# Demo T1 processing notebook

## Configuration options


Please copy the link from OSF to your data:

In [ ]:
# Instead of this step, you could also delete  this cell and the Download data section,
# and instead upload your data directly through the Jupyter Notebook interface. Right-click
# the Jupyter logo (top right) and open in a new tab, then click the upload button in the desired folder.
# If you do this, I recommend you create a data/ folder in the directory of this notebook,
# as would be done during unzipping the OSF data, since this notebook is written with that assumption.

OSF_link = "https://osf.io/3frwb/download/";

Please specify your data type ("Magnitude" or "Complex"):

In [ ]:
dataType = "Magnitude";

In [ ]:
imagePath = '20200229_niloufar_hfmc_NIST/20200229_niloufar_hfmc_NIST_14point.nii.gz'; % Path to image after unzipped

## Download data

In [ ]:
cmd = ['curl -L -o rrsg_dataset.zip', ' ', OSF_link];
[STATUS,MESSAGE] = unix(cmd);
unzip('rrsg_dataset.zip', 'data/');

## Startup qMRLab

In [ ]:
%% MATLAB/OCTAVE CODE
% Adds qMRLab to the path of the environment

try
    cd ../work/qMRLab
    startup
    cd ../../data_submission
catch
    error("qMRLab could not be started correctly.")
end

## Load data

In [ ]:
switch dataType
    case "Magnitude"
        [data, hdr] = nii_load(strcat('data/', imagePath),0,'linear');
        data = double(data);
    case "Complex"
        [data_re, hdr_re] = nii_load(strcat('data/', imagePath{1}),0,'linear');
        [data_im, hdr_im] = nii_load(strcat('data/', imagePath{2}),0,'linear');
        data = double(data_re) + i*double(data_im);
    otherwise
        error("dataType must be either Magnitude or Complex. Stop.")
end

exist('data/')

## Get inversion times

In [ ]:
%get imagePath --from Octave
%get dataType --from Octave
# Strip *.nii.gz file extension
import os
if dataType=="Magnitude":
    tmp1 = os.path.splitext(imagePath)[0]
else:
    tmp1 = os.path.splitext(imagePath[0])[0]
    
image_path_stem = os.path.splitext(tmp1)[0]

# Convert YAML file to JSON
import yaml
import json
with open("data/" + str(image_path_stem) + ".yaml", 'r') as yaml_in, open("data/" + str(image_path_stem) + ".json", "w") as json_out:
    yaml_object = yaml.safe_load(yaml_in) # yaml_object will be a list or a dict
    json.dump(yaml_object, json_out)

In [ ]:
%get image_path_stem --from SoS

json_path = strcat(image_path_stem, '.json');

image_config = loadjson(strcat('data/', json_path));
TI = image_config.sequence.inversion_times;

## Save data and configuration

In [ ]:
save -mat7-binary 'dataAndConfig.mat'

## Fit data

In [ ]:
!sh ../run_ir_rrsg.sh $LD_LIBRARY_PATH ~ /home/jovyan/data_submission/dataAndConfig.mat output.mat

## Save data to nifti

In [ ]:
load('output.mat')

switch dataType
    case "Magnitude"
        hdr.file_name=strcat('t1map.nii');
        nii_save(FitResults.T1,hdr,hdr.file_name);
    case "Complex"
        hdr_re.file_name=strcat('t1map.nii');
        nii_save(FitResults.T1,hdr_re,hdr_re.file_name);
    otherwise
        error("dataType must be either Magnitude or Complex. Stop.")
end


## Plot data

In [ ]:
%% MATLAB/OCTAVE CODE
% Code used to re-orient the images to make pretty figures, and to assign variables with the axis lengths.

T1_map = imrotate(squeeze(FitResults.T1),-90);
xAxis = [0:size(T1_map,2)-1];
yAxis = [0:size(T1_map,1)-1];

% Raw MRI data at different TI values
TI_1 = imrotate(squeeze(abs(data(:,1,:,1))),-90);
TI_2 = imrotate(squeeze(abs(data(:,1,:,2))),-90);
TI_3 = imrotate(squeeze(abs(data(:,1,:,3))),-90);
TI_4 = imrotate(squeeze(abs(data(:,1,:,4))),-90);


In [ ]:
%get T1_map --from Octave
%get TI_1 --from Octave
%get TI_2 --from Octave
%get TI_3 --from Octave
%get TI_4 --from Octave

%get xAxis --from Octave
%get yAxis --from Octave

# PYTHON CODE
# Module imports

import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
config={'showLink': False, 'displayModeBar': False}

init_notebook_mode(connected=True)

from IPython.core.display import display, HTML

from plotly import tools
trace1 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=TI_1,
                   colorscale='Greys',
                   showscale = False,
                   visible=True,
                   name = 'Signal')
trace2 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=TI_2,
                   colorscale='Greys',
                   showscale = False,
                   visible=False,
                   name = 'Signal')
trace3 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=TI_3,
                   colorscale='Greys',
                   showscale = False,
                   visible=False,
                   name = 'Signal')
trace4 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=TI_4,
                   colorscale='Greys',
                   visible=False,
                   showscale = False,
                   name = 'Signal')
trace5 = go.Heatmap(x = xAxis,
                   y = yAxis,
                   z=T1_map,
                   colorscale='Portland',
                   xaxis='x2',
                   yaxis='y2',
                   zmin=0.0,
                   zmax=2500,
                   visible=True,
                   name = 'T1 values (ms)')

data=[trace1, trace2, trace3, trace4, trace5]


updatemenus = list([
    dict(active=0,
         x = 0.12,
         xanchor = 'left',
         y = -0.15,
         yanchor = 'bottom',
         direction = 'up',
         font=dict(
                family='Times New Roman',
                size=16
            ),
         buttons=list([   
            dict(label = '50 ms',
                 method = 'update',
                 args = [{'visible': [True, False, False, False, True]},
                         ]),
            dict(label = '400 ms',
                 method = 'update',
                 args = [{'visible': [False, True, False, False, True]},
                         ]),
            dict(label = '1100 ms',
                 method = 'update',
                 args = [{'visible': [False, False, True, False, True]},
                         ]),
            dict(label = '2400 ms',
                 method = 'update',
                 args = [{'visible': [False, False, False, True, True]},
                         ]),
        ]),
    )
])

layout = dict(
    width=560,
    height=345,
    margin = dict(
                t=40,
                r=50,
                b=10,
                l=50),
    annotations=[
        dict(
            x=0.1,
            y=1.15,
            showarrow=False,
            text='MR Image',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.84,
            y=1.15,
            showarrow=False,
            text='T<sub>1</sub> map',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=1.22,
            y=1.15,
            showarrow=False,
            text='T<sub>1</sub> (ms)',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.02,
            y=-0.15,
            showarrow=False,
            text='TI:',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis = dict(range = [0,255], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0, 0.5]),
    yaxis = dict(range = [0,255], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0.1, 0.9]),
    xaxis2 = dict(range = [0,255], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0.5, 1]),
    yaxis2 = dict(range = [0,255], autorange = False,
             showgrid = False, zeroline = False, showticklabels = False,
             ticks = '', domain=[0.1, 0.9], anchor='x2'),
    showlegend = False,
    autosize = False,
    updatemenus=updatemenus
)


fig = dict(data=data, layout=layout)

iplot(fig, filename = 'basic-heatmap', config = config)